# **智慧医疗赛道：颈椎核磁影像多序列多任务分析**

本代码主要使用不同的多模态大模型对赛题任务进行实现，比较不同大模型的表现，并对其进行优化。

#### **工作流程：**
- 对不同任务编写初始的prompt
- 调用大模型对任务进行实现，输出量化结果
- 尝试不同的大模型，比较不同模型的性能
- Prompt engineering，优化提示词
- 对模型进行微调

#### **优化方向：**
- 任务一、任务二可以用多张图片进行判断
- 如何提高有效样本数（也即按要求输出）

#### **参数预设、导包**

In [2]:
# API参数
API_KEY = "sk-xjkzyprognbbiwogaksfwejwjfpmcttkfhjetreyvhpfqsfs"
API_URL = "https://api.siliconflow.cn/v1"
MODEL_NAME = "Qwen/Qwen2-VL-72B-Instruct"

In [21]:
# 数据路径，先用50个train样本进行测试
train_dir = "D:\\比赛\\通用人工智能大赛\\test_train"  
train_label_dir = "D:\\比赛\\通用人工智能大赛\\cervai_challenge-main\\cervai_challenge-main\\data\\train.json"  

In [1]:
# 导包
from PIL import Image
import io
import base64
import json  
from openai import OpenAI
import os
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

#### **function**

In [28]:
# 图像转webp base64
def convert_image_to_webp_base64(input_image_path):
    try:
        with Image.open(input_image_path) as img:
            byte_arr = io.BytesIO()
            img.save(byte_arr, format='webp')
            byte_arr = byte_arr.getvalue()
            base64_str = base64.b64encode(byte_arr).decode('utf-8')
            return base64_str
    except IOError:
        print(f"Error: Unable to open or convert the image {input_image_path}")
        return None

## **任务一：颈椎曲度评估**

目标：判断颈椎生理曲度状态，分类为：
- 正常（标签 0）
- 曲度变直（标签 1）
- 反弓（标签 2）

说明：颈椎曲度是指颈椎自然的生理弯曲，通常呈前凸（C 形）。曲度评估分为三种：直（生理曲度消失）、正常（前凸曲度良好）、反弓（曲度反向，呈后凸）。

In [5]:
def classify_image(base64_image):
    client = OpenAI(
        api_key = API_KEY, 
        base_url = API_URL
    )

    prompt_text = '''请根据以下设定给出分类结果：
    # 角色：  
    医学影像分析师
    
    # 背景信息：  
    颈椎生理曲度是指颈椎自然的生理弯曲，通常呈前凸（C形）。曲度评估分为三种：直（生理曲度消失）、正常（前凸曲度良好）、反弓（曲度反向，呈后凸）。
    
    # 工作流程/工作任务：  
    1. 接收患者的多序列 MRI 影像。
    2. 分析影像，观察颈椎的生理曲度。
    3. 根据曲度状态，将颈椎生理曲度分类为正常（标签 0）、曲度变直（标签 1）或反弓（标签 2）。
    4. 输出分类结果。
    
    # 输出示例：  
    - 正常（标签 0）
    - 曲度变直（标签 1）
    - 反弓（标签 2）
    
    # 注意事项：  
    在分析影像时，请注意颈椎的解剖结构和曲度特征，确保准确分类。'''

    response = client.chat.completions.create(
        model="Qwen/Qwen2-VL-72B-Instruct",
        messages=[{
            "role": "user",
            "content": [
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{base64_image}"
                    }
                },
                {
                    "type": "text",
                    "text": prompt_text
                }
            ]
        }],
        stream=True
    )

    # 逐块读取结果拼接
    result_text = ""
    for chunk in response:
        if chunk.choices[0].delta.content:
            result_text += chunk.choices[0].delta.content

    # 从结果中提取数字标签
    for label in ['0', '1', '2']:
        if f'标签 {label}' in result_text:
            return int(label)
    return -1  # 未识别

In [6]:
# 加载标签为 dict：{patient_id: 曲度标签}
def load_qd_labels(label_file):
    with open(label_file, 'r', encoding='utf-8') as f:
        raw_list = json.load(f)
    return {item['id']: item['qd'] for item in raw_list}

#### **预测标签**

In [7]:
label_dict = load_qd_labels(train_label_dir)

y_true, y_pred = [], []

for patient_id in sorted(os.listdir(train_dir)):
    sag_dir = os.path.join(train_dir, patient_id, "sag")
    if not os.path.exists(sag_dir):
        continue

    image_files = [f for f in os.listdir(sag_dir) if f.lower().endswith(('.jpg', '.png'))]
    if not image_files:
        continue

    image_path = os.path.join(sag_dir, image_files[0])  # 取第一张图

    print(f"🔍 正在处理 {patient_id}...")
    base64_image = convert_image_to_webp_base64(image_path)
    if not base64_image:
        print(f"[跳过] 无法处理图像：{image_path}")
        continue

    pred_label = classify_image(base64_image)
    true_label = label_dict.get(patient_id, -1)

    print(f"[预测] {patient_id}: 预测={pred_label}, 真值={true_label}")

    if true_label != -1 and pred_label != -1:
        y_true.append(true_label)
        y_pred.append(pred_label)

🔍 正在处理 01240412218008...
[预测] 01240412218008: 预测=0, 真值=1
🔍 正在处理 01240415213025...
[预测] 01240415213025: 预测=0, 真值=2
🔍 正在处理 01240415213035...
[预测] 01240415213035: 预测=0, 真值=2
🔍 正在处理 01240415215019...
[预测] 01240415215019: 预测=1, 真值=1
🔍 正在处理 01240416208050...
[预测] 01240416208050: 预测=0, 真值=1
🔍 正在处理 01240416210045...
[预测] 01240416210045: 预测=0, 真值=2
🔍 正在处理 01240416212030...
[预测] 01240416212030: 预测=2, 真值=1
🔍 正在处理 01240416213015...
[预测] 01240416213015: 预测=0, 真值=1
🔍 正在处理 01240416213047...
[预测] 01240416213047: 预测=0, 真值=1
🔍 正在处理 01240417208061...
[预测] 01240417208061: 预测=1, 真值=1
🔍 正在处理 01240417208068...
[预测] 01240417208068: 预测=0, 真值=1
🔍 正在处理 01240418202036...
[预测] 01240418202036: 预测=1, 真值=0
🔍 正在处理 01240418208067...
[预测] 01240418208067: 预测=1, 真值=1
🔍 正在处理 01240418210053...
[预测] 01240418210053: 预测=0, 真值=1
🔍 正在处理 01240418211022...
[预测] 01240418211022: 预测=0, 真值=1
🔍 正在处理 01240418211046...
[预测] 01240418211046: 预测=2, 真值=0
🔍 正在处理 01240418211057...
[预测] 01240418211057: 预测=0, 真值=0
🔍 正在处理 01240418213069...
[预测] 0

#### **计算指标**

测试指标：准确性、Macro-F1、Weighted-F1

Macro-F1 是对**每个类别的 F1 分数取平均值**，它对每个类别一视同仁，即使某个类别样本很少也会被平等计算。若预测结果偏向某一类，Macro-F1 可能低于 Accuracy。比赛使用的是 Macro-F1 而非 Weighted-F1，是为了鼓励模型在所有类别上表现均衡。

Weighted-F1 也会对每个类别计算 F1 分数，但不是直接平均，而是**按每个类别的样本数量（support）加权平均**。样本多的类别对最终结果贡献更大。

In [10]:
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

# 准确性
acc = accuracy_score(y_true, y_pred)
print(f"\n✅ 任务一（曲度）分类准确率: {acc:.4f} （共 {len(y_true)} 个样本）")

# Macro-F1
macro_f1 = f1_score(y_true, y_pred, average='macro')
print(f"\n✅ 任务一（曲度）分类Macro-F1: {macro_f1:.4f} （共 {len(y_true)} 个样本）")

# Weighted-F1
weighted_f1 = f1_score(y_true, y_pred, average='weighted')
print(f"\n✅ 任务一（曲度）分类Weighted_F1: {weighted_f1:.4f} （共 {len(y_true)} 个样本）\n")

print(classification_report(y_true, y_pred, digits=4))


✅ 任务一（曲度）分类准确率: 0.3333 （共 48 个样本）

✅ 任务一（曲度）分类Macro-F1: 0.2510 （共 48 个样本）

✅ 任务一（曲度）分类Weighted_F1: 0.3808 （共 48 个样本）

              precision    recall  f1-score   support

           0     0.1600    0.5000    0.2424         8
           1     0.8000    0.3750    0.5106        32
           2     0.0000    0.0000    0.0000         8

    accuracy                         0.3333        48
   macro avg     0.3200    0.2917    0.2510        48
weighted avg     0.5600    0.3333    0.3808        48



## **任务二：颈椎顺列评估**

目标：判断颈椎顺列状态，分类为：
- 顺列差（标签 0）
- 顺列可（标签 1）

说明：顺列是指颈椎椎体之间的排列关系。顺列评估分为顺列差（椎体排列不齐，可能存在脱位或滑脱）和顺列可（排列基本正常）。

In [11]:
def classify_sequence(base64_image):
    client = OpenAI(api_key=API_KEY, base_url=API_URL)

    prompt_text = '''你是资深医学影像分析师。请分析这张 T2 矢状位颈椎 MRI 图像，判断椎体之间是否排列整齐。并根据以下分类标准进行判断：
- 顺列差（标签 0）：椎体排列不整齐，可能有滑脱、错位；
- 顺列可（标签 1）：椎体排列基本正常，顺列良好。

请仅输出分类标签（0 或 1），不需要解释或其他内容。'''

    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[{
            "role": "user",
            "content": [
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{base64_image}"
                    }
                },
                {
                    "type": "text",
                    "text": prompt_text
                }
            ]
        }],
        stream=True
    )

    result_text = ""
    for chunk in response:
        if chunk.choices[0].delta.content:
            result_text += chunk.choices[0].delta.content.strip()

    for label in ['0', '1']:
        if label in result_text:
            return int(label)
    return -1  # fallback

In [12]:
# 加载 sl 标签
def load_sl_labels(label_file):
    with open(label_file, 'r', encoding='utf-8') as f:
        raw_list = json.load(f)
    return {item['id']: item['sl'] for item in raw_list}

In [13]:
label_dict = load_sl_labels(train_label_dir)

y_true, y_pred = [], []

for patient_id in sorted(os.listdir(train_dir)):
    sag_dir = os.path.join(train_dir, patient_id, "sag")
    if not os.path.exists(sag_dir):
        continue

    image_files = [f for f in os.listdir(sag_dir) if f.lower().endswith(('.jpg', '.png'))]
    if not image_files:
        continue

    image_path = os.path.join(sag_dir, image_files[0])  # 取第一张图

    print(f"🔍 正在处理 {patient_id}...")
    base64_image = convert_image_to_webp_base64(image_path)
    if not base64_image:
        print(f"[跳过] 无法处理图像：{image_path}")
        continue

    pred_label = classify_sequence(base64_image)
    true_label = label_dict.get(patient_id, -1)

    print(f"[预测] {patient_id}: 预测={pred_label}, 真值={true_label}")

    if true_label != -1 and pred_label != -1:
        y_true.append(true_label)
        y_pred.append(pred_label)


🔍 正在处理 01240412218008...
[预测] 01240412218008: 预测=1, 真值=1
🔍 正在处理 01240415213025...
[预测] 01240415213025: 预测=0, 真值=1
🔍 正在处理 01240415213035...
[预测] 01240415213035: 预测=0, 真值=1
🔍 正在处理 01240415215019...
[预测] 01240415215019: 预测=1, 真值=1
🔍 正在处理 01240416208050...
[预测] 01240416208050: 预测=1, 真值=1
🔍 正在处理 01240416210045...
[预测] 01240416210045: 预测=0, 真值=1
🔍 正在处理 01240416212030...
[预测] 01240416212030: 预测=1, 真值=1
🔍 正在处理 01240416213015...
[预测] 01240416213015: 预测=0, 真值=1
🔍 正在处理 01240416213047...
[预测] 01240416213047: 预测=1, 真值=1
🔍 正在处理 01240417208061...
[预测] 01240417208061: 预测=1, 真值=1
🔍 正在处理 01240417208068...
[预测] 01240417208068: 预测=0, 真值=1
🔍 正在处理 01240418202036...
[预测] 01240418202036: 预测=1, 真值=1
🔍 正在处理 01240418208067...
[预测] 01240418208067: 预测=1, 真值=1
🔍 正在处理 01240418210053...
[预测] 01240418210053: 预测=0, 真值=1
🔍 正在处理 01240418211022...
[预测] 01240418211022: 预测=0, 真值=1
🔍 正在处理 01240418211046...
[预测] 01240418211046: 预测=0, 真值=1
🔍 正在处理 01240418211057...
[预测] 01240418211057: 预测=0, 真值=0
🔍 正在处理 01240418213069...
[预测] 0

In [14]:
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

# 准确性
acc = accuracy_score(y_true, y_pred)
print(f"\n✅ 任务二（顺列）分类准确率: {acc:.4f} （共 {len(y_true)} 个样本）")

# Macro-F1
macro_f1 = f1_score(y_true, y_pred, average='macro')
print(f"\n✅ 任务二（顺列）分类Macro-F1: {macro_f1:.4f} （共 {len(y_true)} 个样本）")

# Weighted-F1
weighted_f1 = f1_score(y_true, y_pred, average='weighted')
print(f"\n✅ 任务二（顺列）分类Weighted_F1: {weighted_f1:.4f} （共 {len(y_true)} 个样本）\n")

print(classification_report(y_true, y_pred, digits=4))


✅ 任务二（顺列）分类准确率: 0.5600 （共 50 个样本）

✅ 任务二（顺列）分类Macro-F1: 0.3969 （共 50 个样本）

✅ 任务二（顺列）分类Weighted_F1: 0.6980 （共 50 个样本）

              precision    recall  f1-score   support

           0     0.0435    1.0000    0.0833         1
           1     1.0000    0.5510    0.7105        49

    accuracy                         0.5600        50
   macro avg     0.5217    0.7755    0.3969        50
weighted avg     0.9809    0.5600    0.6980        50



## **任务三：椎间盘膨突评估**

目标：对 C2-C3 至 C6-C7 共五个椎间位置 进行状态分类：
- 正常（标签 0）
- 膨出（标签 1）
- 突出（标签 2）
- 脱出（标签 3）

**说明：** 颈椎椎间盘膨突是指颈椎间盘的外层变弱或破裂，导致内部物质向外凸出，可能会压迫附近的神经或脊髓。评估分为四种情况：正常（椎间盘没有异常），膨出（椎间盘整体轻微外凸，但外层没有破裂），突出（外层部分破裂，内部物质局部凸出），脱出（外层完全破裂，内部物质可能掉出并移位）。

In [22]:
def classify_zjppt(sag_base64, tra_base64):
    client = OpenAI(api_key=API_KEY, base_url=API_URL)

    prompt = '''请根据以下设定给出MRI图像分类结果：
# 角色：
资深医学影像分析师

# 背景信息：
颈椎中央椎管是指颈椎椎管的中枢部分，包含脊髓、脑脊液及其周围的硬膜等结构，是保护脊髓和神经的重要通道。评估颈椎中央椎管通常分为 0-3 级：0 级表示椎管正常，无狭窄或压迫；1 级为轻度狭窄，脊髓无明显受压；2 级为中度狭窄，脊髓受压但无明显信号改变；3 级为重度狭窄，脊髓明显受压并伴有信号改变。

# 工作任务：
接下来我会给你两张图像，分别为MRI的矢状位图像和对应椎间的横断位图像，请判断该颈椎中央椎管的狭窄状态，并根据上述给你的定义给出分类结果，仅输出数字标签：
- 0 （0级）
- 1 （1级）
- 2 （2级）
- 3 （3级）

# 工作流程务：
5.接收患者的颈椎 MRI 影像。
6.分别分析2张MRI影像，分析颈椎中央椎管的狭窄状态。
7.综合2张图像的结果，将颈椎中央椎管的狭窄状态进行分级。
8.输出颈椎中央椎管的狭窄状态分类结果。'''

    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[{
            "role": "user",
            "content": [
                {"type": "image_url", "image_url": {"url": f"data:image/webp;base64,{sag_base64}"}},
                {"type": "image_url", "image_url": {"url": f"data:image/webp;base64,{tra_base64}"}},
                {"type": "text", "text": prompt}
            ]
        }],
        stream=True
    )

    result_text = ""
    for chunk in response:
        if chunk.choices[0].delta.content:
            result_text += chunk.choices[0].delta.content.strip()

    for label in ['0', '1', '2', '3']:
        if label in result_text:
            return int(label)
    return -1  # fallback


In [23]:
def load_zjppt_labels(json_file):
    with open(json_file, 'r', encoding='utf-8') as f:
        label_list = json.load(f)
    return {item['id']: item['zjppt'] for item in label_list}

In [31]:
CERVICAL_LEVELS = ["2-3", "3-4", "4-5", "5-6", "6-7"]  # 五个椎间位置

label_dict = load_zjppt_labels(train_label_dir)

y_true, y_pred = [], []

for pid in sorted(os.listdir(train_dir)):
    case_dir = os.path.join(train_dir, pid)
    sag_img = os.path.join(case_dir, "sag", "6.png")
    if not os.path.exists(sag_img):
        continue

    try:
        sag_b64 = convert_image_to_webp_base64(sag_img)
    except:
        print(f"❌ 读取矢状图失败: {pid}")
        continue

    for i, level in enumerate(CERVICAL_LEVELS):
        tra_img = os.path.join(case_dir, "tra", f"{level}.png")
        if not os.path.exists(tra_img):
            continue

        try:
            tra_b64 = convert_image_to_webp_base64(tra_img)
            pred = classify_zjppt(sag_b64, tra_b64)
            y_pred.append(pred)
            y_true.append(label_dict.get(pid, [])[i])
            print(f"✔️ {pid} - {level}: pred={pred}, true={label_dict[pid][i]}")
        except Exception as e:
            print(f"⚠️ 失败 {pid} - {level}：{e}")

✔️ 01240412218008 - 2-3: pred=1, true=0
✔️ 01240412218008 - 3-4: pred=3, true=1
✔️ 01240412218008 - 4-5: pred=2, true=1
✔️ 01240412218008 - 5-6: pred=2, true=0
✔️ 01240412218008 - 6-7: pred=1, true=0
✔️ 01240415213025 - 2-3: pred=1, true=2
✔️ 01240415213025 - 3-4: pred=1, true=2
✔️ 01240415213025 - 4-5: pred=1, true=2
✔️ 01240415213025 - 5-6: pred=2, true=2
✔️ 01240415213025 - 6-7: pred=2, true=2
✔️ 01240415213035 - 2-3: pred=2, true=0
✔️ 01240415213035 - 3-4: pred=1, true=2
✔️ 01240415213035 - 4-5: pred=1, true=0
✔️ 01240415213035 - 5-6: pred=2, true=2
✔️ 01240415213035 - 6-7: pred=3, true=2
✔️ 01240415215019 - 2-3: pred=2, true=0
✔️ 01240415215019 - 3-4: pred=1, true=0
✔️ 01240415215019 - 4-5: pred=3, true=1
✔️ 01240415215019 - 5-6: pred=1, true=1
✔️ 01240415215019 - 6-7: pred=1, true=0
✔️ 01240416208050 - 2-3: pred=1, true=2
✔️ 01240416208050 - 3-4: pred=1, true=2
✔️ 01240416208050 - 4-5: pred=1, true=2
✔️ 01240416208050 - 5-6: pred=2, true=2
✔️ 01240416208050 - 6-7: pred=1, true=2


In [32]:
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

# 准确性
acc = accuracy_score(y_true, y_pred)
print(f"\n✅ 任务三（膨突）分类准确率: {acc:.4f} （共 {len(y_true)/5} 个样本）")

# Macro-F1
macro_f1 = f1_score(y_true, y_pred, average='macro')
print(f"\n✅ 任务三（膨突）分类Macro-F1: {macro_f1:.4f} （共 {len(y_true)/5} 个样本）")

# Weighted-F1
weighted_f1 = f1_score(y_true, y_pred, average='weighted')
print(f"\n✅ 任务三（膨突）分类Weighted_F1: {weighted_f1:.4f} （共 {len(y_true)/5} 个样本）\n")

print(classification_report(y_true, y_pred, digits=4))


✅ 任务三（膨突）分类准确率: 0.2200 （共 50.0 个样本）

✅ 任务三（膨突）分类Macro-F1: 0.1209 （共 50.0 个样本）

✅ 任务三（膨突）分类Weighted_F1: 0.2641 （共 50.0 个样本）

              precision    recall  f1-score   support

          -1     0.0000    0.0000    0.0000         0
           0     0.4167    0.0625    0.1087        80
           1     0.0696    0.5789    0.1243        19
           2     0.6290    0.2635    0.3714       148
           3     0.0000    0.0000    0.0000         3

    accuracy                         0.2200       250
   macro avg     0.2231    0.1810    0.1209       250
weighted avg     0.5110    0.2200    0.2641       250



C:\Users\25847\.conda\envs\all\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\25847\.conda\envs\all\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\25847\.conda\envs\all\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## **任务四：中央椎管评估**

目标：对 C2、C2-C3、C3、C3-C4、...、C6-C7、C7 共 11 个位置
进行分级：
- 0 级（标签 0）
- 1 级（标签 1）
- 2 级（标签 2）
- 3 级（标签 3）

**说明：** 颈椎中央椎管是指颈椎椎管的中枢部分，包含脊髓、脑脊液及其周围的硬膜等结构，是保护脊髓和神经的重要通道。评估颈椎中央椎管通常分为 0-3 级：0 级表示椎管正常，无狭窄或压迫；1 级为轻度狭窄，脊髓无明显受压；2 级为中度狭窄，脊髓受压但无明显信号改变；3 级为重度狭窄，脊髓明显受压并伴有信号改变。

In [33]:
def classify_zyzg(sag_base64, tra_base64):
    client = OpenAI(api_key=API_KEY, base_url=API_URL)

    prompt = '''请根据以下设定给出MRI图像分类结果：
# 角色：
资深医学影像分析师

# 背景信息：
颈椎椎间盘膨突是指颈椎间盘的外层变弱或破裂，导致内部物质向外凸出，可能会压迫附近的神经或脊髓。评估分为四种情况：正常（椎间盘没有异常）、膨出（椎间盘整体轻微外凸，但外层没有破裂）、突出（外层部分破裂，内部物质局部凸出）、脱出（外层完全破裂，内部物质可能掉出并移位）。

# 工作任务：
接下来我会给你两张图像，分别为MRI的矢状位图像和对应椎间的横断位图像，请判断该椎间的椎间盘膨突状态，并根据上述给你的定义给出分类结果，仅输出数字标签：
- 0：正常
- 1：膨出
- 2：突出
- 3：脱出

# 工作流程务：
1.接收患者的颈椎 MRI 影像。
2.分别分析3张MRI影像，分析椎间盘状态。
3.综合3张图像的结果，将每个椎间位置分类为正常（标签 0）、膨出（标签 1）、突出（标签 2）或脱出（标签 3）。
4.输出每个椎间位置的状态分类结果。

# 注意事项：
在分析影像时，请注意颈椎椎间盘的状态，确保准确分类。'''

    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[{
            "role": "user",
            "content": [
                {"type": "image_url", "image_url": {"url": f"data:image/webp;base64,{sag_base64}"}},
                {"type": "image_url", "image_url": {"url": f"data:image/webp;base64,{tra_base64}"}},
                {"type": "text", "text": prompt}
            ]
        }],
        stream=True
    )

    result_text = ""
    for chunk in response:
        if chunk.choices[0].delta.content:
            result_text += chunk.choices[0].delta.content.strip()

    for label in ['0', '1', '2', '3']:
        if label in result_text:
            return int(label)
    return -1  # fallback

In [34]:
def load_zyzg_labels(json_file):
    with open(json_file, 'r', encoding='utf-8') as f:
        label_list = json.load(f)
    return {item['id']: item['zyzg'] for item in label_list}

In [35]:
CERVICAL_LEVELS = ["2", "2-3", "3", "3-4", "4", "4-5", "5", "5-6", "6", "6-7", "7"]  # 11个位置

label_dict = load_zyzg_labels(train_label_dir)

y_true, y_pred = [], []

for pid in sorted(os.listdir(train_dir)):
    case_dir = os.path.join(train_dir, pid)
    sag_img = os.path.join(case_dir, "sag", "6.png")
    if not os.path.exists(sag_img):
        continue

    try:
        sag_b64 = convert_image_to_webp_base64(sag_img)
    except:
        print(f"❌ 读取矢状图失败: {pid}")
        continue

    for i, level in enumerate(CERVICAL_LEVELS):
        tra_img = os.path.join(case_dir, "tra", f"{level}.png")
        if not os.path.exists(tra_img):
            continue

        try:
            tra_b64 = convert_image_to_webp_base64(tra_img)
            pred = classify_zyzg(sag_b64, tra_b64)
            y_pred.append(pred)
            y_true.append(label_dict.get(pid, [])[i])
            print(f"✔️ {pid} - {level}: pred={pred}, true={label_dict[pid][i]}")
        except Exception as e:
            print(f"⚠️ 失败 {pid} - {level}：{e}")

✔️ 01240412218008 - 2: pred=1, true=0
✔️ 01240412218008 - 2-3: pred=1, true=0
✔️ 01240412218008 - 3: pred=1, true=0
✔️ 01240412218008 - 3-4: pred=1, true=1
✔️ 01240412218008 - 4: pred=-1, true=0
✔️ 01240412218008 - 4-5: pred=1, true=1
✔️ 01240412218008 - 5: pred=1, true=0
✔️ 01240412218008 - 5-6: pred=0, true=0
✔️ 01240412218008 - 6: pred=1, true=0
✔️ 01240412218008 - 6-7: pred=1, true=0
✔️ 01240412218008 - 7: pred=1, true=0
✔️ 01240415213025 - 2: pred=1, true=0
✔️ 01240415213025 - 2-3: pred=1, true=0
✔️ 01240415213025 - 3: pred=3, true=0
✔️ 01240415213025 - 3-4: pred=1, true=1
✔️ 01240415213025 - 4: pred=2, true=1
✔️ 01240415213025 - 4-5: pred=2, true=2
✔️ 01240415213025 - 5: pred=1, true=1
✔️ 01240415213025 - 5-6: pred=1, true=2
✔️ 01240415213025 - 6: pred=1, true=0
✔️ 01240415213025 - 6-7: pred=3, true=1
✔️ 01240415213025 - 7: pred=1, true=0
✔️ 01240415213035 - 2: pred=1, true=0
✔️ 01240415213035 - 2-3: pred=1, true=0
✔️ 01240415213035 - 3: pred=1, true=0
✔️ 01240415213035 - 3-4: pr

In [36]:
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

# 准确性
acc = accuracy_score(y_true, y_pred)
print(f"\n✅ 任务四（狭窄）分类准确率: {acc:.4f} （共 {len(y_true)/11} 个样本）")

# Macro-F1
macro_f1 = f1_score(y_true, y_pred, average='macro')
print(f"\n✅ 任务四（狭窄）分类Macro-F1: {macro_f1:.4f} （共 {len(y_true)/11} 个样本）")

# Weighted-F1
weighted_f1 = f1_score(y_true, y_pred, average='weighted')
print(f"\n✅ 任务四（狭窄）分类Weighted_F1: {weighted_f1:.4f} （共 {len(y_true)/11} 个样本）\n")

print(classification_report(y_true, y_pred, digits=4))


✅ 任务四（狭窄）分类准确率: 0.2436 （共 50.0 个样本）

✅ 任务四（狭窄）分类Macro-F1: 0.1307 （共 50.0 个样本）

✅ 任务四（狭窄）分类Weighted_F1: 0.2019 （共 50.0 个样本）

              precision    recall  f1-score   support

          -1     0.0000    0.0000    0.0000         0
           0     0.7895    0.0822    0.1489       365
           1     0.2493    0.6714    0.3636       140
           2     0.0990    0.2439    0.1408        41
           3     0.0000    0.0000    0.0000         4

    accuracy                         0.2436       550
   macro avg     0.2276    0.1995    0.1307       550
weighted avg     0.5948    0.2436    0.2019       550



C:\Users\25847\.conda\envs\all\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\25847\.conda\envs\all\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\25847\.conda\envs\all\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
